In [1]:
import re
import subprocess
from datetime import datetime
from functools import lru_cache
from pathlib import Path

# === 使用者設定 ===
INPUT_BIB_PATH = Path('papers.bib')
DEDUP_BIB_PATH = Path('papers_dedup.bib')
HOWPUBLISH_DOCX_PATH = Path('【借名登記】文獻清單匯出.docx')
HOWPUBLISH_SORT_MODE = 'stroke'  # 'stroke' 或 'date'
DATE_SORT_DESCENDING = True      # 僅在 HOWPUBLISH_SORT_MODE='date' 時生效
STROKE_DATA_PATH = Path('reference/Unihan_IRGSources.txt')

# === 工具函式 ===
def clean_entry(entry: str) -> str:
    """刪除值為『未知』『無』或空 {} 的欄位"""
    return re.sub(
        r'^\s*\w+\s*=\s*\{\s*(?:未知|無)?\s*\},?,?\s*$',
        '',
        entry,
        flags=re.MULTILINE
    )

def parse_entry(entry: str):
    """解析 Bib 條目"""
    m_type = re.match(r'(\w+)\s*\{', entry)
    m_title = re.search(r'title\s*=\s*\{(.*?)\}', entry, re.DOTALL)
    m_author = re.search(r'author\s*=\s*\{(.*?)\}', entry, re.DOTALL)
    m_year = re.search(r'year\s*=\s*\{(.*?)\}', entry)
    typ = m_type.group(1) if m_type else 'unknown'
    title = m_title.group(1).strip() if m_title else '未知標題'
    author = m_author.group(1).strip() if m_author else '未知作者'
    year = m_year.group(1).strip() if m_year else '未知年份'
    cleaned_entry = clean_entry(entry)
    return typ, title, author, year, cleaned_entry

def extract_field(entry: str, field_name: str):
    pattern = rf'{field_name}\s*=\s*\{{(.*?)\}}'
    match = re.search(pattern, entry, flags=re.DOTALL | re.IGNORECASE)
    if match:
        value = match.group(1).strip()
        return value or None
    return None

def parse_date_from_text(text: str) -> datetime:
    match = re.search(r'(\d{4})年\s*(\d{1,2})?月?\s*(\d{1,2})?日?', text)
    if not match:
        return datetime.min
    year = int(match.group(1))
    month = int(match.group(2)) if match.group(2) else 1
    day = int(match.group(3)) if match.group(3) else 1
    try:
        return datetime(year, month, day)
    except ValueError:
        return datetime(year, month, 1)

@lru_cache(maxsize=1)
def load_stroke_map():
    stroke_map = {}
    path = STROKE_DATA_PATH
    if not path.exists():
        print(f'⚠️ 找不到字根筆畫檔案：{path}，將以字典順序排序作者。')
        return stroke_map
    with path.open('r', encoding='utf-8') as f:
        for line in f:
            if '\tkTotalStrokes\t' not in line:
                continue
            code, field, value = line.strip().split('\t')
            if field != 'kTotalStrokes':
                continue
            primary = value.split()[0]
            try:
                cp = int(code[2:], 16)
                stroke_map[chr(cp)] = int(primary)
            except ValueError:
                continue
    return stroke_map

def stroke_key(name: str):
    stroke_map = load_stroke_map()
    if not stroke_map:
        return (name,)
    counts = []
    for ch in name:
        if ch.isspace():
            continue
        counts.append(stroke_map.get(ch, 100))
    return tuple(counts) if counts else (100,)

def parse_howpublish_entry(text: str):
    author_segment, _, _ = text.partition('，')
    primary_author = author_segment.split('、')[0].strip() if author_segment else '未知作者'
    date_value = parse_date_from_text(text)
    return {
        'raw': text,
        'primary_author': primary_author,
        'date': date_value,
    }

def collect_howpublish(entries):
    values = []
    for _, _, _, entry in entries:
        howpublish = extract_field(entry, 'howpublished') or extract_field(entry, 'howpublish')
        if howpublish:
            values.append(parse_howpublish_entry(howpublish))
    return values

def sort_howpublish(entries):
    mode = HOWPUBLISH_SORT_MODE.lower()
    if mode == 'date':
        return sorted(
            entries,
            key=lambda item: (item['date'], stroke_key(item['primary_author']), item['raw']),
            reverse=DATE_SORT_DESCENDING
        )
    return sorted(
        entries,
        key=lambda item: (stroke_key(item['primary_author']), item['date'], item['raw'])
    )

def write_howpublish_docx(items, output_path: Path):
    if not items:
        print()
        print('⚠️ 去重後條目未找到 howpublish 欄位，未產生 DOCX。')
        return
    md_path = output_path.with_suffix('.md')
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    sorted_items = sort_howpublish(items)
    sort_label = '作者筆畫' if HOWPUBLISH_SORT_MODE.lower() == 'stroke' else '出版時間'
    total_count = len(sorted_items)
    lines = [
        '# 文獻清單 匯出',
        f'- 產出時間：{timestamp}',
        f'- 排序方式：{sort_label}',
        f'- 條目總數：{total_count}',
        '',
        '## 條目清單',
        ''
    ] + [f"{idx_val}. {item['raw']}" for idx_val, item in enumerate(sorted_items, 1)]

    md_path.write_text('\n'.join(lines), encoding='utf-8')

    subprocess.run([
        'pandoc',
        str(md_path),
        '-f', 'markdown',
        '-t', 'docx',
        '-o', str(output_path)
    ], check=True)

    print()
    print(f"✅ 已輸出 howpublish DOCX（共 {len(sorted_items)} 筆）：{output_path}")

# === 主流程 ===
text = INPUT_BIB_PATH.read_text(encoding='utf-8')
entries = re.split(r'@(?=\w+\s*\{)', text)
entries = [e.strip() for e in entries if e.strip()]
parsed = [parse_entry(e) for e in entries]

# === 去重 ===
seen = {}
duplicates = []
for typ, title, author, year, entry in parsed:
    key = title.lower()
    if key in seen:
        duplicates.append((title, author, year))
    else:
        seen[key] = (title, author, year, entry)
unique_entries = list(seen.values())

# === 統計輸出 ===
total_count = len(parsed)
dup_count = len(duplicates)
unique_count = len(unique_entries)
print(f'原始篇數：{total_count}')
print(f'重複篇數：{dup_count}')
if dup_count:
    print()
    print('重複條目：')
    for title, author, year in duplicates:
        print(f' - {title} / {author} / {year}')
print()
print(f'去重後篇數：{unique_count}')
print('去重後條目：')
for title, author, year, _ in unique_entries:
    print(f' - {title} / {author} / {year}')

# === 寫出去重後 Bib ===
dedup_text = '\n\n'.join(f"@{entry}" for _, _, _, entry in unique_entries)
DEDUP_BIB_PATH.write_text(dedup_text, encoding='utf-8')
print()
print(f"✅ 已輸出去重後檔案：{DEDUP_BIB_PATH}")

# === 匯出 howpublish DOCX ===
howpublish_items = collect_howpublish(unique_entries)
write_howpublish_docx(howpublish_items, HOWPUBLISH_DOCX_PATH)


原始篇數：35
重複篇數：4

重複條目：
 - 借名登記之相關法律效力－最高法院 108 年度台上大字第 1652 號裁定 / 王怡蘋 / 2022
 - 原住民保留地「借名登記契約」的效力－公法規定的私法效力，或私法自治的公法限制？ / 李建良 / 2021
 - 不動產借名登記契約有效性的檢討 / 黃健彰 / 2019
 - 不動產借名登記契約之負擔行為與處分行為 / 蔡旻耿 and 陳靜瑩 / 2016

去重後篇數：31
去重後條目：
 - 不動產借名登記相關民事法律關係－以評析近年最高法院裁判為主 / 黃健彰 / 2025
 - 民事法裁判精選－借名登記之內部關係與外部關係（111 台上 2686 判決） / 顏佑紘 / 2024
 - 論借名登記 / 蔣瑞安 / 2023
 - 借名登記股份的回復與公同共有－最高法院 110 年度台上字第 724 號民事判決 / 陳榮傳 / 2022
 - 原住民保留地的借名登記－大法庭裁定的商榷 / 陳榮傳 / 2022
 - 自耕能力與借名登記－最高法院 107 年度台上字第 1023 號判決評析 / 林慶郎 / 2022
 - 借名登記之舉證責任 / 鄭冠宇 / 2022
 - 借名登記之相關法律效力－最高法院 108 年度台上大字第 1652 號裁定 / 王怡蘋 / 2022
 - 原住民保留地借名登記之法律關係－兼評最高法院 108 年台上大字第 1636 號民事裁定 / 蔡旻耿 / 2022
 - 原住民保留地「借名登記契約」的效力－公法規定的私法效力，或私法自治的公法限制？ / 李建良 / 2021
 - 農地借名登記問題探討－以刪除前土地法第 30 條適用範圍及當事人間契約有效性為核心 / 李維中 / 2021
 - 淺談商標借名登記 / 涂軼 / 2020
 - 不動產借名登記契約有效性的檢討 / 黃健彰 / 2019
 - 我國不動產借名登記後處分之效力問題分析－以最高法院 106 年第 3 次民事庭會議決議為中心 / 陳明燦 / 2018
 - 借名登記契約是否屬「因委任事務之性質不能消滅者」之實務爭議－最高法院 106 年度台上字第 410 號民事判決評析 / 邱玟惠 / 2018
 - 借名登記對外效力之探討－法學方法與債法現代化之思考 / 林大洋 / 2017
 - 借名登記財產之請求返還方